In [ ]:
import geopandas as gpd
import yaml
from common import load_isochrones
from count_amenities_in_accessibility_polygons import (
    count_amenities_in_public_transport_accessibility,
    count_amenities_in_walk_accessibility,
    prepare_amenities,
)

with open("../data/essential_amenities.yaml", "r") as fp:
    amenitiy_categories = list(yaml.safe_load(fp).keys())

In [2]:
CITY = "helsinki"
VERSION = "20250428"

In [ ]:
isochrones = load_isochrones(CITY, VERSION)
amenities = prepare_amenities(CITY)

aciwa = count_amenities_in_walk_accessibility(isochrones, amenities)
aciwa["total"] = aciwa[amenitiy_categories].sum(axis=1)
aciwa.to_csv(
    f"../output/{CITY}/{VERSION}/amenity_counts_in_accessibility.csv",
    index=False,
)

In [ ]:
sgfw = gpd.read_file(f"../output/{CITY}/{VERSION}/stop_geometries_from_walk.geojson")
sgfw["stop_id"] = sgfw["stop_id"].apply(str)

acipta = count_amenities_in_public_transport_accessibility(sgfw, amenities)
acipta["total"] = acipta[amenitiy_categories].sum(axis=1)
acipta.to_csv(
    f"../output/{CITY}/{VERSION}/amenity_counts_in_public_transport_accessibility.csv",
    index=False,
)